### Initialization

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.tensorboard import SummaryWriter

In [ ]:
print(torch.__version__)

device = (
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(f"Using {device} device")

### Data Loading and Preprocessing

In [ ]:
file_path = "train.csv"
data_df = pd.read_csv(file_path)
print(data_df.isnull().sum())

new_data = pd.read_csv(file_path)
print(new_data.isnull().sum().sum() / new_data.size)

data_np = data_df.to_numpy()
mask = ~np.isnan(data_np)
data_np[np.isnan(data_np)] = 0
data = torch.tensor(data_np, dtype=torch.float32).to(device)
mask = torch.tensor(mask, dtype=torch.bool).to(device)

### Generator

In [8]:
class Generator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def forward(self, x):
        return self.model(x)

### Discriminator

In [9]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, x):
        return self.model(x)

### Hyperparameters

In [ ]:
param_grid = {
    'hidden_dim': [16, 32, 64, 128, 1024],
    'lr': [0.1, 0.05, 0.01, 0.0005, 0.0002, 0.0001, 1e-5],
    'batch_size': [16, 32, 64, 128, 256],
    'epochs': [10, 20, 50, 100, 200, 500, 1000, 10000]
}

### Training

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

writer = SummaryWriter("runs/GAN_experiment")

best_params = None
best_score = float('inf')
input_dim = data.size(1)

for hidden_dim in param_grid['hidden_dim']:
    for lr in param_grid['lr']:
        for batch_size in param_grid['batch_size']:
            for epochs in param_grid['epochs']:
                fold_scores = []

                for fold, (train_index, val_index) in enumerate(kf.split(data)):
                    train_data, val_data = data[train_index], data[val_index]
                    train_mask, val_mask = mask[train_index], mask[val_index]

                    generator = Generator(input_dim, hidden_dim).to(device)
                    discriminator = Discriminator(input_dim, hidden_dim).to(device)
                    g_optimizer = optim.Adam(generator.parameters(), lr=lr)
                    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)
                    adversarial_loss = nn.BCEWithLogitsLoss()

                    for epoch in range(epochs):
                        for i in range(0, train_data.size(0), batch_size):
                            real_data = train_data[i:i+batch_size]
                            real_mask = train_mask[i:i+batch_size]
                            generator_input = real_data.clone()
                            generator_input[~real_mask] = 0

                            d_optimizer.zero_grad()
                            fake_data = generator(generator_input)
                            real_loss = -torch.mean(discriminator(real_data))
                            fake_loss = torch.mean(discriminator(fake_data.detach()))
                            d_loss = real_loss + fake_loss
                            d_loss.backward()
                            d_optimizer.step()

                            g_optimizer.zero_grad()
                            g_loss = -torch.mean(discriminator(fake_data))
                            g_loss.backward()
                            g_optimizer.step()

                        writer.add_scalar(f'Fold_{fold}/Generator_Loss', g_loss.item(), epoch)
                        writer.add_scalar(f'Fold_{fold}/Discriminator_Loss', d_loss.item(), epoch)

                    with torch.no_grad():
                        val_generated = generator(val_data.clone())
                        val_completed = val_data.clone()
                        val_completed[~val_mask] = val_generated[~val_mask]
                        val_loss = ((val_completed - val_data) ** 2).mean().item()
                        fold_scores.append(val_loss)

                avg_score = np.mean(fold_scores)
                print(f"Params: hidden_dim={hidden_dim}, lr={lr}, batch_size={batch_size}, epochs={epochs} | Avg Val Loss: {avg_score:.4f}")

                if avg_score < best_score:
                    best_score = avg_score
                    best_params = {
                        'hidden_dim': hidden_dim,
                        'lr': lr,
                        'batch_size': batch_size,
                        'epochs': epochs
                    }
                    torch.save(generator.state_dict(), "generator_best.pth")
                    print(f"Saved the best generator model with loss {best_score:.4f}")

print("Best Hyperparameters:：", best_params)
print("Best Test Loss：", best_score)

writer.close()

### Test

In [ ]:
test_file_path = "test.csv"
test_df = pd.read_csv(test_file_path)

test_np = test_df.to_numpy()
test_mask = ~np.isnan(test_np)
test_np[np.isnan(test_np)] = 0
test_data = torch.tensor(test_np, dtype=torch.float32).to(device)
test_mask = torch.tensor(test_mask, dtype=torch.bool).to(device)

class Generator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def forward(self, x):
        return self.model(x)

#use the tuned hyperparameters
best_params = {
    'hidden_dim': hidden_dim,
    'lr': lr,
    'batch_size': batch_size,
    'epochs': epochs
}
input_dim = test_data.size(1)
hidden_dim = best_params['hidden_dim']

generator = Generator(input_dim, hidden_dim)

model_path = "generator_best.pth"
generator.load_state_dict(torch.load(model_path))
generator.eval()

with torch.no_grad():
    test_input = test_data.clone()
    test_input[~test_mask] = 0
    test_generated = generator(test_input)

    test_completed = test_data.clone()
    test_completed[~test_mask] = test_generated[~test_mask]

completed_df = pd.DataFrame(test_completed.numpy(), columns=test_df.columns)
output_file_path = "completed_test_data.csv"
completed_df.to_csv(output_file_path, index=False)

print(f"Completed test data saved to {output_file_path}")